In [ ]:
# feeder_trace_latest_audit.py  –  dedup HT + audit average (v9)
"""
Workflow
========
1. **Load** `HTCABLE.csv`, drop unused columns, remove fully-identical rows.
2. **Trace** every feeder edge‑by‑edge.
3. **Load** `ENERGYAUDIT.csv`, for each transformer (`FUNC_LOC`) compute
   * `LATEST_DT_DATE`  → most‑recent `SYSTEM_DATE`
   * `DT_LOAD`         → **average** `MD_KVA` across all rows
4. **Merge** audit stats onto trace (`DESTINATION_LOCATION` = `FUNC_LOC`).
5. **Export** `final_two_column.xlsx` with the six requested columns.
"""

from __future__ import annotations
import pandas as pd
from pathlib import Path
from typing import Dict, Tuple, List, Set, Optional

# ── CONFIG ────────────────────────────────────────────────────────────────────
INPUT_HT      = "HTCABLE.csv"
INPUT_ENERGY  = "/media/sagarkumar/New Volume/SAGAR/ENERGYAUDIT.csv"
OUTPUT_PATH   = "final_two_column.xlsx"

FEEDER_ID_COL  = "FEEDERID"
SRC_SWITCH_COL = "SOURCE_SWITCH_ID"
DST_SWITCH_COL = "DESTINATION_SWITCH_ID"
SRC_LOC_COL    = "SOURCE_SSFL"
DST_LOC_COL    = "DESTINATION_SSFL"   # ≡ FUNC_LOC in audit

FUNC_LOC_COL = "FUNC_LOC"
DATE_COL     = "SYSTEM_DATE"
LOAD_COL     = "MD_KVA"

REDUNDANT_COLS = [
    "COMMENTS", "GLOBALID", "MEASUREDLENGTH", "UNNAMED: 0", "OBJECTID"
]
# ─────────────────────────────────────────────────────────────────────────────

# 1️⃣  LOAD & CLEAN HT-CABLE ---------------------------------------------------
ht_path = Path(INPUT_HT).expanduser()
if not ht_path.exists():
    raise FileNotFoundError(ht_path)

ht = (pd.read_excel(ht_path, engine="openpyxl")
      if ht_path.suffix.lower() in {".xls", ".xlsx"}
      else pd.read_csv(ht_path, low_memory=False))
ht.columns = [c.upper() for c in ht.columns]

ht = ht.drop(columns=[c for c in REDUNDANT_COLS if c in ht.columns], errors="ignore")
ht = ht.drop_duplicates()  # remove fully-identical rows

# helper to pull token after 2nd underscore

def _feeder_token(val: str | int | float | None) -> Optional[str]:
    if not isinstance(val, str):
        val = str(val) if val is not None else ""
    p = val.split("_")
    return p[2] if len(p) >= 3 else None

ht["FEEDER_ID"] = ht[FEEDER_ID_COL].apply(_feeder_token)

for col in [SRC_SWITCH_COL, DST_SWITCH_COL, SRC_LOC_COL, DST_LOC_COL]:
    ht[col] = ht[col].astype(str)

edge_cols = [SRC_SWITCH_COL, DST_SWITCH_COL, SRC_LOC_COL, DST_LOC_COL]
source_idx: Dict[Tuple[str, str], pd.DataFrame] = {
    (k[0], k[1]): g[edge_cols]
    for k, g in ht.groupby([SRC_LOC_COL, "FEEDER_ID"], sort=False)
}

# 2️⃣  FEEDER TRACER -----------------------------------------------------------

def trace_feeder(fid: str) -> List[dict]:
    rows: List[dict] = []
    visited: Set[Tuple[str, str]] = set()

    start = ht[(ht[SRC_SWITCH_COL] == fid) & (ht["FEEDER_ID"] == fid)][edge_cols]
    queue = start.to_records(index=False).tolist()

    while queue:
        from_sw, to_sw, src_loc, dst_loc = queue.pop(0)
        if (from_sw, to_sw) in visited:
            continue
        visited.add((from_sw, to_sw))

        rows.append({
            "FEEDER_ID": fid,
            "FROM_TO": f"{from_sw}-{to_sw}",
            "SOURCE_LOCATION": src_loc,
            "DESTINATION_LOCATION": dst_loc,
        })

        nxt = source_idx.get((dst_loc, fid))
        if nxt is not None and not nxt.empty:
            queue.extend(nxt.to_records(index=False).tolist())

    return rows

# 3️⃣  TRACE ALL FEEDERS -------------------------------------------------------
all_edges: List[dict] = []
feeder_ids = [str(f) for f in ht["FEEDER_ID"].dropna().unique()]
print(f"Tracing {len(feeder_ids)} feeders …")
for i, fid in enumerate(feeder_ids, 1):
    if i % 100 == 0 or i in {1, len(feeder_ids)}:
        print(f"  → {i}/{len(feeder_ids)}: {fid}")
    all_edges.extend(trace_feeder(fid))

trace_df = pd.DataFrame(all_edges)

# 4️⃣  LOAD ENERGY-AUDIT & AGGREGATE -----------------------------------------
audit_path = Path(INPUT_ENERGY).expanduser()
if not audit_path.exists():
    raise FileNotFoundError(audit_path)

print("\nLoading energy-audit …")
audit = pd.read_csv(audit_path, low_memory=False, parse_dates=[DATE_COL])
audit.columns = [c.upper() for c in audit.columns]

audit[DATE_COL] = pd.to_datetime(audit[DATE_COL], errors="coerce")

audit = audit[[FUNC_LOC_COL, DATE_COL, LOAD_COL]].dropna(subset=[FUNC_LOC_COL])

agg = (audit.groupby(FUNC_LOC_COL)
           .agg(LATEST_DT_DATE=(DATE_COL, "max"),
                DT_LOAD=(LOAD_COL,  "mean"))
           .reset_index())
agg[FUNC_LOC_COL] = agg[FUNC_LOC_COL].astype(str)

# 5️⃣  MERGE TRACE ← AUDIT -----------------------------------------------------
merged = (trace_df
          .merge(agg, how="left",
                 left_on="DESTINATION_LOCATION",
                 right_on=FUNC_LOC_COL)
          .drop(columns=[FUNC_LOC_COL]))

merged["LATEST_DT_DATE"] = pd.to_datetime(merged["LATEST_DT_DATE"]).dt.date

# 6️⃣  EXPORT ------------------------------------------------------------------
cols = [
    "FEEDER_ID", "FROM_TO",
    "SOURCE_LOCATION", "DESTINATION_LOCATION",
    "LATEST_DT_DATE", "DT_LOAD",
]
merged.to_excel(OUTPUT_PATH, index=False, engine="openpyxl", columns=cols)
print(f"\nSaved {len(merged):,} rows → {OUTPUT_PATH}")

if __name__ == "__main__":
    try:
        from IPython.display import display
        display(merged.head())
    except Exception:
        pass


NEW FILE WITH RANK AND BFS BUT NOT FILTER THE 11KV FEEDER

In [ ]:
# feeder_trace_latest_audit_with_rank.py
"""
Workflow
========
1. Load HTCABLE.csv, drop unused columns, remove fully-identical rows.
2. Trace every feeder edge‑by‑edge, annotate with RANK (distance from feeder start).
3. Load ENERGYAUDIT.csv, for each transformer (FUNC_LOC) compute:
   * LATEST_DT_DATE  → most‑recent SYSTEM_DATE
   * DT_LOAD         → average MD_KVA across all rows
4. Merge audit stats onto trace (DESTINATION_LOCATION = FUNC_LOC).
"""

from __future__ import annotations
import pandas as pd
from pathlib import Path
from typing import Dict, Tuple, List, Set, Optional

# ── CONFIG ────────────────────────────────────────────────────────────────────
INPUT_HT      = "HTCABLE.csv"
INPUT_ENERGY  = "/media/sagarkumar/New Volume/SAGAR/ENERGYAUDIT.csv"
OUTPUT_PATH   = "final_two_column_with_rank.xlsx"

FEEDER_ID_COL  = "FEEDERID"
SRC_SWITCH_COL = "SOURCE_SWITCH_ID"
DST_SWITCH_COL = "DESTINATION_SWITCH_ID"
SRC_LOC_COL    = "SOURCE_SSFL"
DST_LOC_COL    = "DESTINATION_SSFL"   # ≡ FUNC_LOC in audit

FUNC_LOC_COL = "FUNC_LOC"
DATE_COL     = "SYSTEM_DATE"
LOAD_COL     = "MD_KVA"

REDUNDANT_COLS = [
    "COMMENTS", "GLOBALID", "MEASUREDLENGTH", "UNNAMED: 0", "OBJECTID"
]
# ─────────────────────────────────────────────────────────────────────────────

# 1️  LOAD & CLEAN HT-CABLE ---------------------------------------------------
ht_path = Path(INPUT_HT).expanduser()

ht = pd.read_csv(ht_path, low_memory=False)
ht = ht.drop(columns=[c for c in REDUNDANT_COLS if c in ht.columns], errors="ignore")
ht = ht.drop_duplicates()  # remove fully-identical rows

# helper to pull token after 2nd underscore
def _feeder_token(val: str | int | float | None) -> Optional[str]:
    if not isinstance(val, str):
        val = str(val) if val is not None else ""
    p = val.split("_")
    return p[2] if len(p) >= 3 else None

ht["FEEDER_ID"] = ht[FEEDER_ID_COL].apply(_feeder_token)

for col in [SRC_SWITCH_COL, DST_SWITCH_COL, SRC_LOC_COL, DST_LOC_COL]:
    ht[col] = ht[col].astype(str)

edge_cols = [SRC_SWITCH_COL, DST_SWITCH_COL, SRC_LOC_COL, DST_LOC_COL]
source_idx: Dict[Tuple[str, str], pd.DataFrame] = {
    (k[0], k[1]): g[edge_cols]
    for k, g in ht.groupby([SRC_LOC_COL, "FEEDER_ID"], sort=False)
}

# 2️  FEEDER TRACER (with RANK) -----------------------------------------------
def trace_feeder(fid: str) -> List[dict]:
    rows: List[dict] = []
    visited: Set[Tuple[str, str]] = set()

    # queue holds tuples: (edge_tuple, rank)
    start = ht[(ht[SRC_SWITCH_COL] == fid) & (ht["FEEDER_ID"] == fid)][edge_cols]
    queue = [(row, 0) for row in start.to_records(index=False).tolist()]  # (edge, rank)

    while queue:
        (from_sw, to_sw, src_loc, dst_loc), rank = queue.pop(0)
        if (from_sw, to_sw) in visited:
            continue
        visited.add((from_sw, to_sw))

        rows.append({
            "FEEDER_ID": fid,
            "FROM_TO": f"{from_sw}-{to_sw}",
            "SOURCE_LOCATION": src_loc,
            "DESTINATION_LOCATION": dst_loc,
            "RANK": rank  # Level in the feeder tree
        })

        nxt = source_idx.get((dst_loc, fid))
        if nxt is not None and not nxt.empty:
            # Each downstream edge gets rank+1
            queue.extend([(row, rank + 1) for row in nxt.to_records(index=False).tolist()])

    return rows

# 3️  TRACE ALL FEEDERS -------------------------------------------------------
all_edges: List[dict] = []
feeder_ids = [str(f) for f in ht["FEEDER_ID"].dropna().unique()]
print(f"Tracing {len(feeder_ids)} feeders …")
for i, fid in enumerate(feeder_ids, 1):
    if i % 100 == 0 or i in {1, len(feeder_ids)}:
        print(f"  → {i}/{len(feeder_ids)}: {fid}")
    all_edges.extend(trace_feeder(fid))

trace_df = pd.DataFrame(all_edges)

# 4️ LOAD ENERGY-AUDIT & AGGREGATE -----------------------------------------
audit_path = Path(INPUT_ENERGY).expanduser()
if not audit_path.exists():
    raise FileNotFoundError(audit_path)

print("\nLoading energy-audit …")
audit = pd.read_csv(audit_path, low_memory=False, parse_dates=[DATE_COL])
audit.columns = [c.upper() for c in audit.columns]

audit[DATE_COL] = pd.to_datetime(audit[DATE_COL], errors="coerce")

audit = audit[[FUNC_LOC_COL, DATE_COL, LOAD_COL]].dropna(subset=[FUNC_LOC_COL])

agg = (audit.groupby(FUNC_LOC_COL)
           .agg(LATEST_DT_DATE=(DATE_COL, "max"),
                DT_LOAD=(LOAD_COL,  "mean"))
           .reset_index())
agg[FUNC_LOC_COL] = agg[FUNC_LOC_COL].astype(str)

# 5️  MERGE TRACE ← AUDIT -----------------------------------------------------
merged = (trace_df.merge(agg, how="left",
                 left_on="DESTINATION_LOCATION",
                 right_on=FUNC_LOC_COL).drop(columns=[FUNC_LOC_COL]))

merged["LATEST_DT_DATE"] = pd.to_datetime(merged["LATEST_DT_DATE"]).dt.date

# 6️  EXPORT ------------------------------------------------------------------
cols = ["FEEDER_ID", "FROM_TO", "SOURCE_LOCATION", "DESTINATION_LOCATION", "RANK", "LATEST_DT_DATE", "DT_LOAD"]
merged.to_excel(OUTPUT_PATH, index=False, engine="openpyxl", columns=cols)
print(f"\nSaved {len(merged):,} rows → {OUTPUT_PATH}")

if __name__ == "__main__":
    try:
        from IPython.display import display
        display(merged.head())
    except Exception:
        pass


Tracing 1764 feeders …
  → 1/1764: 15454
  → 100/1764: 220AAR108
  → 200/1764: 25326
  → 300/1764: 11920
  → 400/1764: 18619
  → 500/1764: 40920
  → 600/1764: 03717
  → 700/1764: 31265
  → 800/1764: 28672
  → 900/1764: 35870
  → 1000/1764: 29562
  → 1100/1764: 18638
  → 1200/1764: 40301
  → 1300/1764: 34726
  → 1400/1764: 220SAK12
  → 1500/1764: 34673
  → 1600/1764: 31981
  → 1700/1764: 30956
  → 1764/1764: BUSPT

Loading energy-audit …

Saved 17,319 rows → final_two_column_with_rank.xlsx


,FEEDER_ID,FROM_TO,SOURCE_LOCATION,DESTINATION_LOCATION,RANK,LATEST_DT_DATE,DT_LOAD
0,15454,15454-38196,1S-MH-MU-ZST-RSTN-24TH,1S-MH-MU-ZST-CL02-1238,0,2025-04-04,127.425882
1,15454,38195-34116,1S-MH-MU-ZST-CL02-1238,1S-MH-MU-ZST-CL02-0894,1,2025-04-04,233.628927
2,15454,38197-DT,1S-MH-MU-ZST-CL02-1238,1S-MH-MU-ZST-CL02-1238,1,2025-04-04,127.425882
3,15454,34114-32764,1S-MH-MU-ZST-CL02-0894,1S-MH-MU-ZST-CL01-0860,2,2025-04-04,134.062123
4,15454,34115-DT,1S-MH-MU-ZST-CL02-0894,1S-MH-MU-ZST-CL02-0894,2,2025-04-04,233.628927


FILTER 11KV BUT NOT INCLUDE ALL THE VALUE OF 11KV BECAUSE OF 11Kv, 11KV, 11kV

In [1]:
# feeder_trace_latest_audit_with_rank.py
"""
Workflow
========
1. Load HTCABLE.csv, drop unused columns, remove fully-identical rows.
2. Trace every feeder edge‑by‑edge, annotate with RANK (distance from feeder start).
3. Load ENERGYAUDIT.csv, for each transformer (FUNC_LOC) compute:
   * LATEST_DT_DATE  → most‑recent SYSTEM_DATE
   * DT_LOAD         → average MD_KVA across all rows
4. Merge audit stats onto trace (DESTINATION_LOCATION = FUNC_LOC).
"""

from __future__ import annotations
import pandas as pd
from pathlib import Path
from typing import Dict, Tuple, List, Set, Optional

# ── CONFIG ────────────────────────────────────────────────────────────────────
INPUT_HT      = "HTCABLE.csv"
INPUT_ENERGY  = "/media/sagarkumar/New Volume/SAGAR/ENERGYAUDIT.csv"
OUTPUT_PATH   = "final_two_column_with_rank_11.xlsx"

FEEDER_ID_COL  = "FEEDERID"
SRC_SWITCH_COL = "SOURCE_SWITCH_ID"
DST_SWITCH_COL = "DESTINATION_SWITCH_ID"
SRC_LOC_COL    = "SOURCE_SSFL"
DST_LOC_COL    = "DESTINATION_SSFL"   # ≡ FUNC_LOC in audit

FUNC_LOC_COL = "FUNC_LOC"
DATE_COL     = "SYSTEM_DATE"
LOAD_COL     = "MD_KVA"

REDUNDANT_COLS = [
    "COMMENTS", "GLOBALID", "MEASUREDLENGTH", "UNNAMED: 0", "OBJECTID"
]
# ─────────────────────────────────────────────────────────────────────────────

# 1️  LOAD & CLEAN HT-CABLE ---------------------------------------------------
ht_path = Path(INPUT_HT).expanduser()

ht = pd.read_csv(ht_path, low_memory=False)
ht = ht.drop(columns=[c for c in REDUNDANT_COLS if c in ht.columns], errors="ignore")
ht = ht.drop_duplicates()  # remove fully-identical rows

# helper to pull token after 2nd underscore
def _feeder_token(val: str | int | float | None) -> Optional[str]:
    if not isinstance(val, str):
        val = str(val) if val is not None else ""
    p = val.split("_")
    return p[2] if len(p) >= 3 and p[1] == '11kV' else None





# def extract_feeder_id(value: str | int | float | None) -> Optional[str]:
#     """
#     Return the token after the 2nd underscore only if the middle token is '11kV'.
#     Example: 'AMBVLI_11kV_19556' ➜ '19556'
#     """
#     if not isinstance(value, str):
#         value = str(value) if value is not None else ""
#     parts = value.split("_")
#     if len(parts) >= 3 :
#         return parts[2]
#     return None



ht["FEEDER_ID"] = ht[FEEDER_ID_COL].apply(_feeder_token)

for col in [SRC_SWITCH_COL, DST_SWITCH_COL, SRC_LOC_COL, DST_LOC_COL]:
    ht[col] = ht[col].astype(str)

edge_cols = [SRC_SWITCH_COL, DST_SWITCH_COL, SRC_LOC_COL, DST_LOC_COL]
source_idx: Dict[Tuple[str, str], pd.DataFrame] = {
    (k[0], k[1]): g[edge_cols]
    for k, g in ht.groupby([SRC_LOC_COL, "FEEDER_ID"], sort=False)
}

# 2️  FEEDER TRACER (with RANK) -----------------------------------------------
def trace_feeder(fid: str) -> List[dict]:
    rows: List[dict] = []
    visited: Set[Tuple[str, str]] = set()

    # queue holds tuples: (edge_tuple, rank)
    start = ht[(ht[SRC_SWITCH_COL] == fid) & (ht["FEEDER_ID"] == fid)][edge_cols]
    queue = [(row, 0) for row in start.to_records(index=False).tolist()]  # (edge, rank)

    while queue:
        (from_sw, to_sw, src_loc, dst_loc), rank = queue.pop(0)
        if (from_sw, to_sw) in visited:
            continue
        visited.add((from_sw, to_sw))

        rows.append({
            "FEEDER_ID": fid,
            "FROM_TO": f"{from_sw}-{to_sw}",
            "SOURCE_LOCATION": src_loc,
            "DESTINATION_LOCATION": dst_loc,
            "RANK": rank  # Level in the feeder tree
        })

        nxt = source_idx.get((dst_loc, fid))
        if nxt is not None and not nxt.empty:
            # Each downstream edge gets rank+1
            queue.extend([(row, rank + 1) for row in nxt.to_records(index=False).tolist()])

    return rows

# 3️  TRACE ALL FEEDERS -------------------------------------------------------
all_edges: List[dict] = []
feeder_ids = [str(f) for f in ht["FEEDER_ID"].dropna().unique()]
print(f"Tracing {len(feeder_ids)} feeders …")
for i, fid in enumerate(feeder_ids, 1):
    if i % 100 == 0 or i in {1, len(feeder_ids)}:
        print(f"  → {i}/{len(feeder_ids)}: {fid}")
    all_edges.extend(trace_feeder(fid))

trace_df = pd.DataFrame(all_edges)

# 4️ LOAD ENERGY-AUDIT & AGGREGATE -----------------------------------------
audit_path = Path(INPUT_ENERGY).expanduser()
if not audit_path.exists():
    raise FileNotFoundError(audit_path)

print("\nLoading energy-audit …")
audit = pd.read_csv(audit_path, low_memory=False, parse_dates=[DATE_COL])
audit.columns = [c.upper() for c in audit.columns]

audit[DATE_COL] = pd.to_datetime(audit[DATE_COL], errors="coerce")

audit = audit[[FUNC_LOC_COL, DATE_COL, LOAD_COL]].dropna(subset=[FUNC_LOC_COL])

agg = (audit.groupby(FUNC_LOC_COL)
           .agg(LATEST_DT_DATE=(DATE_COL, "max"),
                DT_LOAD=(LOAD_COL,  "mean"))
           .reset_index())
agg[FUNC_LOC_COL] = agg[FUNC_LOC_COL].astype(str)

# 5️  MERGE TRACE ← AUDIT -----------------------------------------------------
merged = (trace_df.merge(agg, how="left",
                 left_on="DESTINATION_LOCATION",
                 right_on=FUNC_LOC_COL).drop(columns=[FUNC_LOC_COL]))

merged["LATEST_DT_DATE"] = pd.to_datetime(merged["LATEST_DT_DATE"]).dt.date

# 6️  EXPORT ------------------------------------------------------------------
cols = ["FEEDER_ID", "FROM_TO", "SOURCE_LOCATION", "DESTINATION_LOCATION", "RANK", "LATEST_DT_DATE", "DT_LOAD"]
merged.to_excel(OUTPUT_PATH, index=False, engine="openpyxl", columns=cols)
print(f"\nSaved {len(merged):,} rows → {OUTPUT_PATH}")

if __name__ == "__main__":
    try:
        from IPython.display import display
        display(merged.head())
    except Exception:
        pass


Tracing 1192 feeders …
  → 1/1192: 15451
  → 100/1192: 43878
  → 200/1192: 31002
  → 300/1192: 35414
  → 400/1192: 01514
  → 500/1192: 30965
  → 600/1192: 07825
  → 700/1192: 26106
  → 800/1192: 14414
  → 900/1192: 03132
  → 1000/1192: 41516
  → 1100/1192: 41750
  → 1192/1192: BUSPT

Loading energy-audit …

Saved 14,045 rows → final_two_column_with_rank_11.xlsx


,FEEDER_ID,FROM_TO,SOURCE_LOCATION,DESTINATION_LOCATION,RANK,LATEST_DT_DATE,DT_LOAD
0,15451,15451-31551,1S-MH-MU-ZST-RSTN-24TH,1S-MH-MU-ZST-CL02-0885,0,2025-04-04,372.357927
1,15451,31553-39603,1S-MH-MU-ZST-CL02-0885,1S-MH-MU-ZST-RSTN-KHAR,1,2025-04-04,102.345161
2,15451,39786-39785,1S-MH-MU-ZST-CL02-0885,1S-MH-MU-ZST-CL02-0414,1,2025-04-04,442.128986
3,15451,31554-DT,1S-MH-MU-ZST-CL02-0885,1S-MH-MU-ZST-CL02-0885,1,2025-04-04,372.357927
4,15451,15770-29785,1S-MH-MU-ZST-CL02-0414,1S-MH-MU-ZST-CL02-0198,2,2025-04-04,536.223301


In [7]:
# feeder_trace_latest_audit_with_rank_updated.py
"""
Workflow
========
1. Load HTCABLE.csv, drop unused columns, remove fully‑identical rows.
2. Trace every feeder edge‑by‑edge, annotate with RANK (distance from feeder start).
3. Load ENERGYAUDIT.csv, for each transformer (FUNC_LOC) compute:
   * LATEST_DT_DATE  → most‑recent SYSTEM_DATE
   * DT_LOAD         → average MD_KVA across all rows
4. Merge audit stats onto trace (DESTINATION_LOCATION = FUNC_LOC).
5. Create LOCATION column identical to DESTINATION_LOCATION for easier downstream joins.
"""

from __future__ import annotations
import pandas as pd
from pathlib import Path
from typing import Dict, Tuple, List, Set, Optional

# ── CONFIG ────────────────────────────────────────────────────────────────────
INPUT_HT      = "HTCABLE.csv"
INPUT_ENERGY  = "/media/sagarkumar/New Volume/SAGAR/ENERGYAUDIT.csv"
OUTPUT_PATH   = "final_two_column_with_rank_11_full.csv"

FEEDER_ID_COL  = "FEEDERID"
SRC_SWITCH_COL = "SOURCE_SWITCH_ID"
DST_SWITCH_COL = "DESTINATION_SWITCH_ID"
SRC_LOC_COL    = "SOURCE_SSFL"
DST_LOC_COL    = "DESTINATION_SSFL"   # ≡ FUNC_LOC in audit

FUNC_LOC_COL = "FUNC_LOC"
DATE_COL     = "SYSTEM_DATE"
LOAD_COL     = "MD_KVA"

REDUNDANT_COLS = [
    "COMMENTS", "GLOBALID", "MEASUREDLENGTH", "UNNAMED: 0", "OBJECTID"
]
# ─────────────────────────────────────────────────────────────────────────────

# 1️  LOAD & CLEAN HT-CABLE ---------------------------------------------------
ht_path = Path(INPUT_HT).expanduser()

ht = pd.read_csv(ht_path, low_memory=False)
ht = ht.drop(columns=[c for c in REDUNDANT_COLS if c in ht.columns], errors="ignore")
ht = ht.drop_duplicates()  # remove fully-identical rows

# helper to pull token after 2nd underscore
def _feeder_token(val: str | int | float | None) -> Optional[str]:
    if not isinstance(val, str):
        val = str(val) if val is not None else ""
    p = val.split("_")
    return p[2] if len(p) >= 3 and (p[1] == '11kV' or p[1]=='11Kv' or p[1]=='11KV') else None

ht["FEEDER_ID"] = ht[FEEDER_ID_COL].apply(_feeder_token)

for col in [SRC_SWITCH_COL, DST_SWITCH_COL, SRC_LOC_COL, DST_LOC_COL]:
    ht[col] = ht[col].astype(str)

edge_cols = [SRC_SWITCH_COL, DST_SWITCH_COL, SRC_LOC_COL, DST_LOC_COL]
source_idx: Dict[Tuple[str, str], pd.DataFrame] = {
    (k[0], k[1]): g[edge_cols]
    for k, g in ht.groupby([SRC_LOC_COL, "FEEDER_ID"], sort=False)
}

# 2️  FEEDER TRACER (with RANK) -----------------------------------------------
def trace_feeder(fid: str) -> List[dict]:
    rows: List[dict] = []
    visited: Set[Tuple[str, str]] = set()

    # queue holds tuples: (edge_tuple, rank)
    start = ht[(ht[SRC_SWITCH_COL] == fid) & (ht["FEEDER_ID"] == fid)][edge_cols]
    queue = [(row, 0) for row in start.to_records(index=False).tolist()]  # (edge, rank)

    while queue:
        (from_sw, to_sw, src_loc, dst_loc), rank = queue.pop(0)
        if (from_sw, to_sw) in visited:
            continue
        visited.add((from_sw, to_sw))

        rows.append({
            "FEEDER_ID": fid,
            "FROM_TO": f"{from_sw}-{to_sw}",
            "SOURCE_LOCATION": src_loc,
            "DESTINATION_LOCATION": dst_loc,
            "RANK": rank  # Level in the feeder tree
        })

        nxt = source_idx.get((dst_loc, fid))
        if nxt is not None and not nxt.empty:
            # Each downstream edge gets rank+1
            queue.extend([(row, rank + 1) for row in nxt.to_records(index=False).tolist()])

    return rows

# 3️  TRACE ALL FEEDERS -------------------------------------------------------
all_edges: List[dict] = []
feeder_ids = [str(f) for f in ht["FEEDER_ID"].dropna().unique()]
print(f"Tracing {len(feeder_ids)} feeders …")
for i, fid in enumerate(feeder_ids, 1):
    if i % 100 == 0 or i in {1, len(feeder_ids)}:
        print(f"  → {i}/{len(feeder_ids)}: {fid}")
    all_edges.extend(trace_feeder(fid))

trace_df = pd.DataFrame(all_edges)

# 4️ LOAD ENERGY-AUDIT & AGGREGATE -----------------------------------------
audit_path = Path(INPUT_ENERGY).expanduser()
if not audit_path.exists():
    raise FileNotFoundError(audit_path)

print("\nLoading energy-audit …")
audit = pd.read_csv(audit_path, low_memory=False, parse_dates=[DATE_COL])
audit.columns = [c.upper() for c in audit.columns]

audit[DATE_COL] = pd.to_datetime(audit[DATE_COL], errors="coerce")

audit = audit[[FUNC_LOC_COL, DATE_COL, LOAD_COL]].dropna(subset=[FUNC_LOC_COL])

agg = (audit.groupby(FUNC_LOC_COL)
           .agg(LATEST_DT_DATE=(DATE_COL, "max"),
                DT_LOAD=(LOAD_COL,  "mean"))
           .reset_index())
agg[FUNC_LOC_COL] = agg[FUNC_LOC_COL].astype(str)

# 5️  MERGE TRACE ← AUDIT -----------------------------------------------------
merged = (trace_df.merge(agg, how="left",
                 left_on="DESTINATION_LOCATION",
                 right_on=FUNC_LOC_COL).drop(columns=[FUNC_LOC_COL]))

merged["LATEST_DT_DATE"] = pd.to_datetime(merged["LATEST_DT_DATE"]).dt.date

# NEW COLUMN: LOCATION identical to DESTINATION_LOCATION
merged["LOCATION"] = merged["DESTINATION_LOCATION"]

# 6️  EXPORT ------------------------------------------------------------------
cols = [
    "FEEDER_ID", "FROM_TO", "SOURCE_LOCATION", "DESTINATION_LOCATION", "LOCATION",
    "RANK", "LATEST_DT_DATE", "DT_LOAD"
]
merged.to_csv(OUTPUT_PATH, index=False, columns=cols)
print(f"\nSaved {len(merged):,} rows  {OUTPUT_PATH}")

if __name__ == "__main__":
        from IPython.display import display

Tracing 1424 feeders …
  → 1/1424: 15454
  → 100/1424: 41897
  → 200/1424: 28223
  → 300/1424: 39624
  → 400/1424: 41709
  → 500/1424: 31267
  → 600/1424: 28674
  → 700/1424: 35873
  → 800/1424: 30135
  → 900/1424: 18093
  → 1000/1424: 30031
  → 1100/1424: 03101
  → 1200/1424: 35038
  → 1300/1424: 30886
  → 1400/1424: 19090
  → 1424/1424: BUSPT

Loading energy-audit …

Saved 16,541 rows  final_two_column_with_rank_11_full.csv


In [2]:
def extract_mid_token(val: str | int | float | None) -> Optional[str]:
    if not isinstance(val, str):
        val = str(val) if val is not None else ""
    p = val.split("_")
    return p[1] if len(p) >= 2 else None

unique_mid_tokens = ht[FEEDER_ID_COL].apply(extract_mid_token).unique()
print("Unique mid tokens from FEEDER_ID_COL:", unique_mid_tokens)

Unique mid tokens from FEEDER_ID_COL: [None '11KV' '11kV' 'GOR0552' 'BOR00552' 'BOR00952' 'SAK1152' 'SAK0452'
 '33KV' '33kV' '22kV' 'GOI00152' '11Kv' 'REACTOR' '22KV' 'VER00152'
 'GHO01452' 'GHO01652' 'AAR01352' 'AAR01552' 'GOI00352' 'GOI00852'
 'CHE00952' 'CHE00152' 'CHE00552' 'CHE00652' 'DHN00352' '40973' '40974'
 '40976' '40977' '40978' '40980' '33360' 'AAR00452' 'AAR00552' 'AAR00652'
 'AAR00852' 'AAR01152' 'BOR00152' 'BOR00752' 'CHE00252' 'CHE00852'
 'CHE01052' 'GHD00752' 'GHD00952' 'GHD1052' 'GOI00252' 'GOI00752'
 'GOR00252' 'GOR00752' 'GOR01052' 'SAK00752' 'SAK00952' 'SAK01252'
 'VER00652' 'VER00752' 'VER00852' 'VER01052' 'VER01352' '33Kv' 'BOR00352'
 'BOR01652' 'VER01452' 'VER01552' 'SAK00652' 'SAK00352' 'GOI00652'
 'GOR00452' 'GOR00152']


In [3]:
def extract_mid_token(val: str | int | float | None) -> Optional[str]:
    if not isinstance(val, str):
        val = str(val) if val is not None else ""
    p = val.split("_")
    return p[0] if len(p) >= 2 else None

unique_mid_tokens = ht[FEEDER_ID_COL].apply(extract_mid_token).unique()
print("Unique mid tokens from FEEDER_ID_COL:", unique_mid_tokens)

Unique mid tokens from FEEDER_ID_COL: [None '24THRD' 'AAR01752' 'AAR01852' 'AAR01952' 'AAR1252' 'AAR1452' 'ACRO'
 'AMBVLI' 'ANDHRI' 'ANIK' 'ARY220' 'ARY' 'BANDRA' 'BBLWDI' 'BHAVANS'
 'BHAYW' 'BHYNDR' 'BKC' 'BNDRTE' 'BORIVLI' 'BORVLI' 'BOR' 'CAMA' 'CHAKALA'
 'CHBNDR' 'CHDNGR' 'CHDVLI' 'CHE' 'CHMBUR' 'CHMBU' 'CHMB' 'CHUNA' 'CHVALI'
 'CPWDMAREC' 'DAHICHNKA' 'DAHISRW' 'DAHISR' 'DEVIDAS' 'DHA' 'DINDO'
 'ERANGL' 'ESIC' 'GHO00452' 'GHOD' 'GKLDHM' 'GNSHNG' 'GODREJBKC'
 'GOI00452' 'GOI220' 'GOR220' 'GORAI' 'GOREG' 'HCC' 'HINGWALA' 'HIRANANDA'
 'HIRANA' 'HULL' 'JANKALYAN' 'JBNGR' 'JUHUN' 'JUHU' 'KADAMWADI' 'KALANR'
 'KALINA' 'KALPATARU' 'KANA' 'KANDI' 'KHAR' 'KIE' 'KOHINR' 'KURLA' 'KURL'
 'LKHWLA' 'MAHANANDA' 'MAHULSRA' 'MAKERS' 'MALAD' 'MAL' 'MANK' 'MAROL'
 'MBI00152' 'MBI00652' 'MBO00152' 'MBR00152' 'MBR00252' 'MBR00552'
 'MBR00752' 'MBR00852' 'MGHWDI' 'MHADAMANK' 'MHADASAH' 'MHADSAH' 'MIDC'
 'MINDSP' 'MIRA' 'MMRDA' 'MNR00152' 'MTR00152' 'MTR00252' 'MTR00352'
 'MVR00152' 'NAHAR SHAKTI DSS' 'NA

In [6]:
def extract_mid_token(val: str | int | float | None) -> Optional[str]:
    if not isinstance(val, str):
        val = str(val) if val is not None else ""
    p = val.split("_")
    return p[2] if len(p) >= 3 else None

unique_mid_tokens = ht[FEEDER_ID_COL].apply(extract_mid_token).unique()
print("Unique mid tokens from FEEDER_ID_COL:", unique_mid_tokens)

Unique mid tokens from FEEDER_ID_COL: [None '15454' '15451' ... '40877' '40878' 'BUSPT']


REMOVE DT AND USE 11KV VOATGE FILE


In [53]:
# feeder_trace_latest_audit_with_rank_updated.py
"""
Workflow
========
1. Load HTCABLE.csv, drop unused columns, remove fully‑identical rows.
2. Trace every feeder edge‑by‑edge, annotate with RANK (distance from feeder start).
3. Load ENERGYAUDIT.csv, for each transformer (FUNC_LOC) compute:
   * LATEST_DT_DATE  → most‑recent SYSTEM_DATE
   * DT_LOAD         → average MD_KVA across all rows
4. Merge audit stats onto trace (DESTINATION_LOCATION = FUNC_LOC).
"""

from __future__ import annotations
import pandas as pd
from pathlib import Path
from typing import Dict, Tuple, List, Set, Optional
import re

# ── CONFIG ────────────────────────────────────────────────────────────────────
INPUT_HT = "/media/sagark24/New Volume/MERGE CDIS/2-Year-data/HTCABLE_Clean.csv"
INPUT_ENERGY = "/media/sagark24/New Volume/MERGE CDIS/2-Year-data/ENERGYAUDIT.csv"
# FEEDER_LIST_PATH = "/media/sagark24/New Volume/MERGE CDIS/2-Year-data/FEEDERDETAILS.csv"
OUTPUT_PATH = "/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/final_two_column_with_rank_11_withoutDT_another_feeder.csv"

FEEDER_ID_COL  = "FEEDERID"
SRC_SWITCH_COL = "SOURCE_SWITCH_ID"
DST_SWITCH_COL = "DESTINATION_SWITCH_ID"
SRC_LOC_COL    = "SOURCE_SSFL"
DST_LOC_COL    = "DESTINATION_SSFL"   # ≡ FUNC_LOC in audit

FUNC_LOC_COL = "FUNC_LOC"
DATE_COL     = "SYSTEM_DATE"
LOAD_COL     = "MD_KVA"

REDUNDANT_COLS = [

]
# ─────────────────────────────────────────────────────────────────────────────

# 1️  LOAD & CLEAN HT-CABLE ---------------------------------------------------
ht_path = Path(INPUT_HT).expanduser()

ht = pd.read_csv(ht_path, low_memory=False)
ht = ht.drop(columns=[c for c in REDUNDANT_COLS if c in ht.columns], errors="ignore")
ht = ht.drop_duplicates()  # remove fully-identical rows

# helper to pull token after 2nd underscore
def _feeder_token(val: str | int | float | None) -> Optional[str]:
    if not isinstance(val, str):
        val = str(val) if val is not None else ""
    p = val.split("_")
    return p[2] if len(p) >= 3 and (p[1] == '11kV' or p[1]=='11Kv' or p[1]=='11KV')  else None

ht["FEEDER_ID"] = ht[FEEDER_ID_COL].apply(_feeder_token)

for col in [SRC_SWITCH_COL, DST_SWITCH_COL, SRC_LOC_COL, DST_LOC_COL]:
    ht[col] = ht[col].astype(str)

edge_cols = [SRC_SWITCH_COL, DST_SWITCH_COL, SRC_LOC_COL, DST_LOC_COL]
source_idx: Dict[Tuple[str, str], pd.DataFrame] = {
    (k[0], k[1]): g[edge_cols]
    for k, g in ht.groupby([SRC_LOC_COL, "FEEDER_ID"], sort=False)
}

# 2️  FEEDER TRACER (with RANK) -----------------------------------------------
def trace_feeder(fid: str) -> List[dict]:
    rows: List[dict] = []
    visited: Set[Tuple[str, str]] = set()

    # queue holds tuples: (edge_tuple, rank)
    start = ht[(ht[SRC_SWITCH_COL] == fid) & (ht["FEEDER_ID"] == fid)][edge_cols]
    queue = [(row, 0) for row in start.to_records(index=False).tolist()]  # (edge, rank)

    while queue:
        (from_sw, to_sw, src_loc, dst_loc), rank = queue.pop(0)
        if (from_sw, to_sw) in visited:
            continue
        visited.add((from_sw, to_sw))

        rows.append({
            "FEEDER_ID": fid,
            "FROM_TO": f"{from_sw}-{to_sw}",
            "SOURCE_LOCATION": src_loc,
            "DESTINATION_LOCATION": dst_loc,
            "RANK": rank  # Level in the feeder tree
        })

        nxt = source_idx.get((dst_loc, fid))
        if nxt is not None and not nxt.empty:
            # Each downstream edge gets rank+1
            queue.extend([(row, rank + 1) for row in nxt.to_records(index=False).tolist()])

    return rows

# 3️  TRACE ALL FEEDERS -------------------------------------------------------
all_edges: List[dict] = []
feeder_ids = [str(f) for f in ht["FEEDER_ID"].dropna().unique()]
print(f"Tracing {len(feeder_ids)} feeders …")
for i, fid in enumerate(feeder_ids, 1):
    if i % 100 == 0 or i in {1, len(feeder_ids)}:
        print(f"  → {i}/{len(feeder_ids)}: {fid}")
    all_edges.extend(trace_feeder(fid))

trace_df = pd.DataFrame(all_edges)

# 4️ LOAD ENERGY-AUDIT & AGGREGATE -----------------------------------------
audit_path = Path(INPUT_ENERGY).expanduser()
if not audit_path.exists():
    raise FileNotFoundError(audit_path)

print("\nLoading energy-audit …")
audit = pd.read_csv(audit_path, low_memory=False, parse_dates=[DATE_COL])
audit.columns = [c.upper() for c in audit.columns]

audit[DATE_COL] = pd.to_datetime(audit[DATE_COL], errors="coerce")

audit = audit[[FUNC_LOC_COL, DATE_COL, LOAD_COL]].dropna(subset=[FUNC_LOC_COL])

agg = (audit.groupby(FUNC_LOC_COL)
           .agg(LATEST_DT_DATE=(DATE_COL, "max"),
                DT_LOAD=(LOAD_COL,  "mean"))
           .reset_index())
agg[FUNC_LOC_COL] = agg[FUNC_LOC_COL].astype(str)

# 5️  MERGE TRACE ← AUDIT -----------------------------------------------------
merged = (trace_df.merge(agg, how="left",
                 left_on="DESTINATION_LOCATION",
                 right_on=FUNC_LOC_COL).drop(columns=[FUNC_LOC_COL]))

merged["LATEST_DT_DATE"] = pd.to_datetime(merged["LATEST_DT_DATE"]).dt.date

# Add LOCATION column as a copy of DESTINATION_LOCATION
merged["LOCATION"] = merged["DESTINATION_LOCATION"]

# KEEP ONLY ROWS WHERE FROM_TO IS xxxx-yyyy BOTH NUMERIC
def from_to_is_numeric(s):
    match = re.fullmatch(r'(\d+)-(\d+)', str(s))
    return bool(match)
merged = merged[merged['FROM_TO'].apply(from_to_is_numeric)]

# 6️  EXPORT ------------------------------------------------------------------
cols = ["FEEDER_ID", "FROM_TO", "SOURCE_LOCATION", "DESTINATION_LOCATION", "LOCATION", "RANK", "LATEST_DT_DATE", "DT_LOAD"]
merged.to_csv(OUTPUT_PATH, index=False, columns=cols)
print(f"\nSaved {len(merged):,} rows → {OUTPUT_PATH}")

if __name__ == "__main__":
    try:
        from IPython.display import display
        display(merged.head())
    except Exception:
        pass 

Tracing 1424 feeders …
  → 1/1424: 15454
  → 100/1424: 41897
  → 200/1424: 28223
  → 300/1424: 39624
  → 400/1424: 41709
  → 500/1424: 31267
  → 600/1424: 28674
  → 700/1424: 35873
  → 800/1424: 30135
  → 900/1424: 18093
  → 1000/1424: 30031
  → 1100/1424: 03101
  → 1200/1424: 35038
  → 1300/1424: 30886
  → 1400/1424: 19090
  → 1424/1424: BUSPT

Loading energy-audit …

Saved 7,333 rows → /media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/final_two_column_with_rank_11_withoutDT_another_feeder.csv


,FEEDER_ID,FROM_TO,SOURCE_LOCATION,DESTINATION_LOCATION,RANK,LATEST_DT_DATE,DT_LOAD,LOCATION
0,15454,15454-38196,1S-MH-MU-ZST-RSTN-24TH,1S-MH-MU-ZST-CL02-1238,0,2025-04-04,127.425882,1S-MH-MU-ZST-CL02-1238
1,15454,38195-34116,1S-MH-MU-ZST-CL02-1238,1S-MH-MU-ZST-CL02-0894,1,2025-04-04,233.628927,1S-MH-MU-ZST-CL02-0894
3,15454,34114-32764,1S-MH-MU-ZST-CL02-0894,1S-MH-MU-ZST-CL01-0860,2,2025-04-04,134.062123,1S-MH-MU-ZST-CL01-0860
5,15454,32766-31556,1S-MH-MU-ZST-CL01-0860,1S-MH-MU-ZST-CL02-0815,3,2025-04-04,245.587090,1S-MH-MU-ZST-CL02-0815
7,15454,31555-4467,1S-MH-MU-ZST-CL02-0815,1S-MH-MU-ZST-CL02-0054,4,2025-04-04,364.485990,1S-MH-MU-ZST-CL02-0054


FEEDER FROM THE ANOTHER FILE

In [79]:
df = pd.read_csv('/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/final_two_column_with_rank_11_withoutDT.csv')
col = df['FEEDER_ID'].dropna().unique()
print(len(col))

945
